https://towardsdatascience.com/interaction-effect-in-multiple-regression-3091a5d0fadd

In [82]:
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = 'all'

In [83]:
import pandas as pd

pd.set_option( 'display.max_columns' , None ) 

In [84]:
import pandas as pd

In [85]:
url = 'https://raw.githubusercontent.com/ContinuumIO/PythonVIS2013/master/data/auto-mpg.csv'

df = pd.read_csv( filepath_or_buffer = url )

In [86]:
df.head()

,mpg,cyl,displ,hp,weight,accel,yr,origin,name
0,18.0,8,307.0,130,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140,3449,10.5,70,1,ford torino


In [87]:
df.shape

(392, 9)

In [88]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 392 entries, 0 to 391
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   mpg     392 non-null    float64
 1   cyl     392 non-null    int64  
 2   displ   392 non-null    float64
 3   hp      392 non-null    int64  
 4   weight  392 non-null    int64  
 5   accel   392 non-null    float64
 6   yr      392 non-null    int64  
 7   origin  392 non-null    int64  
 8   name    392 non-null    object 
dtypes: float64(3), int64(5), object(1)
memory usage: 27.7+ KB


In [89]:
df.drop( 'name' , axis = 1 , inplace = True )

df.head()

,mpg,cyl,displ,hp,weight,accel,yr,origin
0,18.0,8,307.0,130,3504,12.0,70,1
1,15.0,8,350.0,165,3693,11.5,70,1
2,18.0,8,318.0,150,3436,11.0,70,1
3,16.0,8,304.0,150,3433,12.0,70,1
4,17.0,8,302.0,140,3449,10.5,70,1


In [90]:
for col in df.columns:
    df[ col ] = pd.to_numeric( df[ col ] , errors = 'coerce' )
    
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 392 entries, 0 to 391
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   mpg     392 non-null    float64
 1   cyl     392 non-null    int64  
 2   displ   392 non-null    float64
 3   hp      392 non-null    int64  
 4   weight  392 non-null    int64  
 5   accel   392 non-null    float64
 6   yr      392 non-null    int64  
 7   origin  392 non-null    int64  
dtypes: float64(3), int64(5)
memory usage: 24.6 KB


In [91]:
df.isna().sum()

mpg       0
cyl       0
displ     0
hp        0
weight    0
accel     0
yr        0
origin    0
dtype: int64

In [92]:
X = df.drop( 'mpg' , axis = 1 )

y = df[ 'mpg' ]

In [93]:
from statsmodels.regression import linear_model

In [94]:
mdl = linear_model.OLS( y , X ).fit()

In [95]:
mdl.pvalues

cyl       2.975467e-02
displ     4.318785e-03
hp        1.776088e-03
weight    1.481849e-18
accel     3.788147e-01
yr        4.169891e-79
origin    4.296529e-06
dtype: float64

In [96]:
alpha = 0.05

mdl.pvalues < 0.05

cyl        True
displ      True
hp         True
weight     True
accel     False
yr         True
origin     True
dtype: bool

> **accel coefficient non significatif**

*In the above model summary, we can see that except acceleration, all other features have a p-value less than 0.05 and are statistically significant. Even if acceleration standalone is not helpful in the prediction of mpg, we are interested in finding out whether acceleration after interacting with other variables is having an effect on mpg. Also, we are interested to know the presence of all significant interaction terms.*

In [97]:
from sklearn.preprocessing import PolynomialFeatures

In [98]:
poly_tsf = PolynomialFeatures( 2 , interaction_only = True , include_bias = False )

poly_tsf.fit( X )

print( poly_tsf.get_feature_names() )

X_interaction = poly_tsf.transform( X )

print( X.shape )

print( X_interaction.shape )

PolynomialFeatures(include_bias=False, interaction_only=True)

['x0', 'x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x0 x1', 'x0 x2', 'x0 x3', 'x0 x4', 'x0 x5', 'x0 x6', 'x1 x2', 'x1 x3', 'x1 x4', 'x1 x5', 'x1 x6', 'x2 x3', 'x2 x4', 'x2 x5', 'x2 x6', 'x3 x4', 'x3 x5', 'x3 x6', 'x4 x5', 'x4 x6', 'x5 x6']
(392, 7)
(392, 28)


- x0 : cyl       
- x1 : displ     
- x2 : hp        
- x3 : weight    
- x4 : accel     
- x5 : yr        
- x6 : origin    

In [99]:
import itertools

In [101]:
list( X.columns ) + list( map( lambda x : x[0] + '-' + x[1] , list( itertools.combinations( X.columns , 2 ) ) ) )

['cyl',
 'displ',
 'hp',
 'weight',
 'accel',
 'yr',
 'origin',
 'cyl-displ',
 'cyl-hp',
 'cyl-weight',
 'cyl-accel',
 'cyl-yr',
 'cyl-origin',
 'displ-hp',
 'displ-weight',
 'displ-accel',
 'displ-yr',
 'displ-origin',
 'hp-weight',
 'hp-accel',
 'hp-yr',
 'hp-origin',
 'weight-accel',
 'weight-yr',
 'weight-origin',
 'accel-yr',
 'accel-origin',
 'yr-origin']

In [102]:
cols_interaction = list( X.columns ) + list( map( lambda x : x[0] + '-' + x[1] , list( itertools.combinations( X.columns , 2 ) ) ) )

df_interaction  = pd.DataFrame( X_interaction, columns = cols_interaction )

In [103]:
df_interaction.head()

,cyl,displ,hp,weight,accel,yr,origin,cyl-displ,cyl-hp,cyl-weight,cyl-accel,cyl-yr,cyl-origin,displ-hp,displ-weight,displ-accel,displ-yr,displ-origin,hp-weight,hp-accel,hp-yr,hp-origin,weight-accel,weight-yr,weight-origin,accel-yr,accel-origin,yr-origin
0,8.0,307.0,130.0,3504.0,12.0,70.0,1.0,2456.0,1040.0,28032.0,96.0,560.0,8.0,39910.0,1075728.0,3684.0,21490.0,307.0,455520.0,1560.0,9100.0,130.0,42048.0,245280.0,3504.0,840.0,12.0,70.0
1,8.0,350.0,165.0,3693.0,11.5,70.0,1.0,2800.0,1320.0,29544.0,92.0,560.0,8.0,57750.0,1292550.0,4025.0,24500.0,350.0,609345.0,1897.5,11550.0,165.0,42469.5,258510.0,3693.0,805.0,11.5,70.0
2,8.0,318.0,150.0,3436.0,11.0,70.0,1.0,2544.0,1200.0,27488.0,88.0,560.0,8.0,47700.0,1092648.0,3498.0,22260.0,318.0,515400.0,1650.0,10500.0,150.0,37796.0,240520.0,3436.0,770.0,11.0,70.0
3,8.0,304.0,150.0,3433.0,12.0,70.0,1.0,2432.0,1200.0,27464.0,96.0,560.0,8.0,45600.0,1043632.0,3648.0,21280.0,304.0,514950.0,1800.0,10500.0,150.0,41196.0,240310.0,3433.0,840.0,12.0,70.0
4,8.0,302.0,140.0,3449.0,10.5,70.0,1.0,2416.0,1120.0,27592.0,84.0,560.0,8.0,42280.0,1041598.0,3171.0,21140.0,302.0,482860.0,1470.0,9800.0,140.0,36214.5,241430.0,3449.0,735.0,10.5,70.0


In [104]:
mdl_interaction = linear_model.OLS( y , df_interaction ).fit()

In [105]:
alpha = 0.05 

print( mdl_interaction.pvalues[ mdl_interaction.pvalues < alpha ] )

displ           3.235311e-03
hp              2.344125e-02
accel           5.225950e-04
yr              6.936438e-13
origin          3.907030e-03
cyl-yr          3.532847e-02
displ-yr        5.495075e-03
hp-accel        3.484642e-02
hp-yr           4.453001e-02
accel-yr        1.483186e-02
accel-origin    4.562421e-03
dtype: float64


*It is important to note that in the example above, the p-value of acceleration is high but it is included in interaction terms. In such a case, we have to include the main effects of acceleration in the model i.e. the coefficient of acceleration even when it is not statistically significant due to the hierarchy principle. Hierarchy principle states that if there are two features X₁ and X₂ in an interaction term, we have to include both of their coefficients(β₁ and β₂) in the model even when the p-values associated to them are very high.*